In [1]:
import pandas as pd
import geopandas as gpd
import mio
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def tri_array(p0, p1, p2, area, px, py):
    p0x, p0y = p0
    p1x, p1y = p1
    p2x, p2y = p2
    
    # do the calc, using stolen formula
    s = 1/(2*area)*(p0y*p2x - p0x*p2y + (p2y - p0y)*px + (p0x - p2x)*py)
    t = 1/(2*area)*(p0x*p1y - p0y*p1x + (p0y - p1y)*px + (p1x - p0x)*py)
    u = 1 - s -t
    r = (s>0) & (t>0) & (u>0)
    
    return r.astype(int)

In [3]:
def signed_area(p0, p1, p2):
        # calc signed area of triangle
        p0x, p0y = p0
        p1x, p1y = p1
        p2x, p2y = p2
        
        a = 0.5 *(-p1y*p2x + p0y*(-p1x + p2x) + p0x*(p1y - p2y) + p1x*p2y)

        return a

In [4]:
def create_empty(p0, p1, p2):
    
    xmin = int(min(p0[0], p1[0], p2[0])) -1
    xmax = int(max(p0[0], p1[0], p2[0])) +1
    ymin = int(min(p0[1], p1[1], p2[1])) -1
    ymax = int(max(p0[1], p1[1], p2[1])) +1
    df = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    xa = np.arange(xmin, xmax,  1)
    ya = np.arange(ymax, ymin, -1)
    px, py = np.meshgrid(xa, ya)
    return xa, ya, px, py

In [5]:
def get_df_list(tab):
    print(f'reading {tab}', end=' ')
    vec = gpd.read_file(tab)
    ldf = []
    count_ok = 0
    count_er = 0
    
    i = 0
    print(len(vec) / 1e3, 'x1000 triangles')
    print('create mini rasters')
    for ind, row in vec.iterrows():
        i += 1
        mio.show_perc(i, len(vec), 10000)
        p0 = (row.x0, row.y0)
        p1 = (row.x1, row.y1)
        p2 = (row.x2, row.y2)

        xa, ya, px, py = create_empty(p0, p1, p2)

        area = signed_area(p0, p1, p2)

        if area != 0:
            count_ok += 1
            ar = tri_array(p0, p1, p2, area, px, py)
            df = pd.DataFrame(ar, columns=xa, index=ya)
            if len(df) > 0:
                ldf.append(df)
        else:
            # propably vertical
            count_er += 1
    return ldf

In [6]:
def get_empty(ldf):
    lx, ly = [], []
    for df in ldf:
        lx.append(df.columns.min())
        lx.append(df.columns.max())
        ly.append(df.index.min())
        ly.append(df.index.max())
    xmin, xmax = min(lx)-5, max(lx)+5
    ymin, ymax = min(ly)-5, max(ly)+5
    
    empty = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    return empty

In [ ]:
# MAIN
tag = '1091-13'
ldf = get_df_list(f'result/{tag}.tab')
big = get_empty(ldf)
# combine all the mini rasters
for i, small in enumerate(ldf):
    mio.show_perc(i, len(ldf), 10_000)
    xmin, xmax = small.columns.min(), small.columns.max()
    ymin, ymax = small.index.min(), small.index.max()
    big.loc[ymax:ymin, xmin:xmax] = np.maximum(big.loc[ymax:ymin, xmin:xmax], small)
mio.write_raster(big.fillna(0).astype('uint8'), f'result/{tag}_raster.tif',color_map={0:(0,0,0),1:(255,0,0)})

reading result/1091-13.tab 345.7 x1000 triangles
create mini rasters
2.89% 5.79% 8.68% 11.57% 14.46% 17.36% 20.25% 23.14% 26.03% 28.93% 